In [53]:
import torch
from torch import nn 
import numpy as np
from scipy.special import softmax
import random

In [59]:
x = [100] + [0]*40
baseline(np.array(x), s=50, e=90,factor = 0)

[50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]


-344.00000000000006

In [57]:
def arrival2available(x, s, e, total_vehicles=100, num_step=96):
    x = softmax(x)
    arrival_time = generate_arrival(x)
    available_vehicles = [0 for _ in range(num_step)]
    arrival_cnt = [arrival_time.count(i) for i in range(num_step)]
    for i in range(total_vehicles):
        for t in range(arrival_time[i], num_step):
            available_vehicles[t] += 1
    return available_vehicles, arrival_cnt


def baseline(x, s, e, factor=0.05, flag=1):
    if flag != 3:
        x = softmax(x)
        arrival_time = generate_arrival(x, s=s, e=e)
    elif flag == 3:
        arrival_time = x
    print(arrival_time)
    dataset = generate_dataset_test(arrival_time)
    arrival_time = dataset['arrival_time']
    depart_time = dataset['depart_time']
    initial_state = dataset['initial_state']
    required_energy = dataset['required_energy']
    final_energy = dataset['final_energy']
    return -ES(factor, arrival_time, depart_time, initial_state, required_energy, final_energy, flag=flag)

def generate_arrival(x0, s=31, e=80, total_vehicles=100):
    arrival = [idx+s for idx,x in enumerate(x0) for _ in range(int(x*total_vehicles))]
    c = total_vehicles - len(arrival)
    if c <= 0:
        arrival = arrival[:total_vehicles]
    else:
        x1 = sorted(zip(x0, list(range(len(x0)))), key=lambda x: -x[0]) 
        idxs = [b for a,b in x1]
        while len(idxs) < c and c > 0:
            c -= len(idxs)
            arrival += list(range(s,e+1))            
        if c > 0 and len(idxs) >= c:
            arrival += [i+s for i in idxs[:c]]
    #print(min(arrival), max(arrival))
    return arrival

def generate_dataset_test(arrival_time, num_step=96, total_vehicles=100, battery_capacity = 8, lam=12, seed=0, initial_state=None):
    np.random.seed(seed)
    random.seed(seed)
    arrival_time = np.sort(arrival_time)
    depart_time = np.ones_like(arrival_time)*num_step
    if initial_state == None: # 20~30%
        initial_state = np.random.uniform(0.2*8, 0.3*8, size=(total_vehicles,))
    else:
        initial_state = initial_state
    #required_energy=np.random.uniform(2.0, 6.0, size=(total_vehicles,))
    final_energy =  0.9 * battery_capacity * np.ones((total_vehicles))
    required_energy = np.round(final_energy-np.array(initial_state), 2)
    #print(np.sum(required_energy))
    dataset = {'arrival_time':arrival_time, 'depart_time':depart_time, 'initial_state':initial_state, 'required_energy': required_energy, 'final_energy':final_energy}
    return dataset


def ES(factor, arrival_time, depart_time, initial_state, required_energy, final_energy, power_capacity=8, max_power=0.6, decay_rate=0.06, num_steps=96, total_vehicles=100,flag=1):
    initial_state_EDF=np.copy(initial_state)
    u_mat=np.zeros((total_vehicles, num_steps), dtype=float)

    #-5 to avoid computation infeasibility at this time
    for t in range(int(arrival_time[0]), num_steps-2):
        power_budget=power_capacity #Change this for variable case

        #Firstly get the states
        vehicle_ending_index = (arrival_time < t).sum()
        step_initial_SOC = np.copy(initial_state_EDF[:vehicle_ending_index])
        final_energy_needed = np.copy(final_energy[:vehicle_ending_index])
        depart_schedule=np.copy(depart_time[:vehicle_ending_index])
        u_val=np.zeros_like(step_initial_SOC)
        index=np.argsort(depart_schedule) #sort the departure time
        charging_sessions=0

        num_active_sessions=0
        for i in range(vehicle_ending_index):
            if depart_schedule[i] >= t:
                if step_initial_SOC[i] < final_energy[i]-1e-3:
                    num_active_sessions += 1
        #print("Number of active sessions", num_active_sessions)
        if num_active_sessions == 0:
            num_active_sessions = 1
        shared_power = power_capacity / num_active_sessions

        for i in range(vehicle_ending_index):
            if depart_schedule[i]>=t:
                if step_initial_SOC[i] < final_energy[i]:
                    u_val[i] = shared_power

        updated_val = np.minimum(u_val, np.ones_like(u_val) * max_power - decay_rate * step_initial_SOC)
        updated_val = np.minimum(updated_val, final_energy_needed-step_initial_SOC) #newly added
        initial_state_EDF[:vehicle_ending_index] += updated_val
        #print(f't:{t}, vehicle_ending_indx:{vehicle_ending_index}, updated_val:{len(updated_val)}, u_mat:{len(u_mat[:vehicle_ending_index,t])},{len(u_mat[:52,t])}')
        u_mat[:vehicle_ending_index, t] = updated_val
    stay_time = np.sum(depart_time) - np.sum(arrival_time)
    if flag == 1: return np.sum(u_mat) - factor * stay_time
    else: return u_mat